In [ ]:
import tensorflow as tf
#import numpy as np
import pandas as pd
#from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [ ]:
df = pd.read_csv("Real estate.csv")
df.head()
df

In [ ]:
print(df.info())
print(df.describe())

In [ ]:
df.drop('No', axis=1,inplace=True)

#normalising

df.distanceToTheNearestMRTStation = (df['distanceToTheNearestMRTStation'] - df['distanceToTheNearestMRTStation'].mean()) / (df['distanceToTheNearestMRTStation'].max() - df['distanceToTheNearestMRTStation'].min())
df.latitude = (df['latitude'] - df['latitude'].mean()) / (df['latitude'].max() - df['latitude'].min())
df.longitude = (df['longitude'] - df['longitude'].mean()) / (df['longitude'].max() - df['longitude'].min())
df.transactionDate = df['transactionDate'].astype('int')
df.number_of_convenience_stores = (df['number_of_convenience_stores'] - df['number_of_convenience_stores'].mean()) / (df['number_of_convenience_stores'].max() - df['number_of_convenience_stores'].min())
df.houseAge = (df['houseAge'] - df['houseAge'].mean()) / (df['houseAge'].max() - df['houseAge'].min())


#creating dommie variables

dummies = pd.get_dummies(df['transactionDate'], dtype='int')
df = pd.concat([df,dummies],axis=1) 
df.drop('transactionDate', axis=1, inplace=True)

df = df.dropna()

df.head()

In [ ]:
#spliting inti training and evaluation set
train, val = train_test_split(df, test_size=0.2)

# training dataset
train_target = train.pop('house_price_of_unit_area')
train_dataset = tf.data.Dataset.from_tensor_slices((train.values, train_target.values))

#evaluation set

val_target = val.pop('house_price_of_unit_area')
val_dataset = tf.data.Dataset.from_tensor_slices((val.values, val_target.values))


#sample from training set
for feat, targ in train_dataset.take(1):
     print ('Features: {}, Target: {}'.format(feat, targ))

In [ ]:
#model

model = keras.Sequential()
model.add(layers.Dense(32, input_shape=(1,))) # shape is (1,) because can't use batches due to less data
model.add(layers.Dense(16, activation='relu') )
model.add(layers.Dense(8, activation='relu' ))
model.add(layers.Dense(2, activation='relu' ))
model.add(layers.Dense(1))
          


In [ ]:
optimizer = tf.keras.optimizers.RMSprop(0.001)

model.compile(loss='mse',optimizer=optimizer,metrics=['accuracy'])


In [ ]:
model.summary()


In [ ]:
history = model.fit( train_dataset,validation_data=val_dataset ,epochs=200, verbose=1)
